📥 1. Load dan Preprocessing Dataset IMDB

In [40]:
from tensorflow import keras
from keras.preprocessing.sequence import pad_sequences

# Load dataset: 25.000 data latih, 25.000 data test
(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data(num_words=10000)

# Batasi panjang review maksimal 200 kata
maxlen = 200
X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


🧠 2. Model LSTM Sederhana (Tanpa Attention)

In [41]:
model_lstm = keras.models.Sequential([
    keras.layers.Embedding(input_dim=10000, output_dim=32, input_length=maxlen),
    keras.layers.LSTM(32),
    keras.layers.Dense(1, activation="sigmoid")
])

model_lstm.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
model_lstm.fit(X_train, y_train, epochs=3, validation_split=0.2, batch_size=64)

Epoch 1/3


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - accuracy: 0.6647 - loss: 0.5902 - val_accuracy: 0.8510 - val_loss: 0.3651
Epoch 2/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.8859 - loss: 0.2898 - val_accuracy: 0.8478 - val_loss: 0.3590
Epoch 3/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.9217 - loss: 0.2119 - val_accuracy: 0.8598 - val_loss: 0.3273


🌟 3. Tambahkan Attention Layer (Manual)

In [44]:
class Attention(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__()
        self.dense = tf.keras.layers.Dense(1)  # dibuat hanya sekali, di awal

    def call(self, inputs):
        # inputs: (batch_size, time_steps, hidden_dim)
        score = self.dense(inputs)  # (batch_size, time_steps, 1)
        attention_weights = tf.nn.softmax(score, axis=1)  # softmax over time
        context_vector = attention_weights * inputs
        context_vector = tf.reduce_sum(context_vector, axis=1)  # (batch_size, hidden_dim)
        return context_vector

In [45]:
inputs = keras.Input(shape=(maxlen,))
x = keras.layers.Embedding(input_dim=10000, output_dim=32)(inputs)
x = keras.layers.Bidirectional(keras.layers.LSTM(32, return_sequences=True))(x)
x = Attention()(x)
outputs = keras.layers.Dense(1, activation="sigmoid")(x)

model_attention = keras.Model(inputs, outputs)
model_attention.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
model_attention.fit(X_train, y_train, epochs=3, validation_split=0.2, batch_size=64)

Epoch 1/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 20ms/step - accuracy: 0.6868 - loss: 0.5522 - val_accuracy: 0.8728 - val_loss: 0.3197
Epoch 2/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 17ms/step - accuracy: 0.9047 - loss: 0.2496 - val_accuracy: 0.8718 - val_loss: 0.3140
Epoch 3/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.9298 - loss: 0.1877 - val_accuracy: 0.8766 - val_loss: 0.3120


📊 4. Evaluasi Model

In [46]:
loss, acc = model_attention.evaluate(X_test, y_test)
print("Akurasi pada data uji:", acc)

782/782 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.8647 - loss: 0.3319
Akurasi pada data uji: 0.8636000156402588


🔍 5. Visualisasi Prediksi

In [47]:
import numpy as np

# Ambil review pertama
sample = X_test[0:1]
prediction = model_attention.predict(sample)[0][0]
sentiment = "Positif" if prediction >= 0.5 else "Negatif"

print(f"Prediksi Sentimen: {sentiment} ({prediction:.2f})")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 215ms/step
Prediksi Sentimen: Negatif (0.02)


## ✅ Catatan:

* `Embedding`: mengubah ID kata menjadi vektor.
* `LSTM` atau `Bidirectional LSTM` menangkap konteks sekuens.
* **Attention** fokus pada bagian penting dari kalimat (misalnya kata “not”, “terrible”).
* Model bisa dikembangkan untuk NER, QA, dan Summarization dengan konsep yang sama.
